In [1]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('sigfest/database-for-emotion-recognition-system-gameemo', path='gameemo', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/sigfest/database-for-emotion-recognition-system-gameemo


In [1]:
import os

print(os.listdir("gameemo"))


['GAMEEMO']


In [2]:
print(os.listdir("gameemo/GAMEEMO"))


['(S01)', '(S02)', '(S03)', '(S04)', '(S05)', '(S06)', '(S07)', '(S08)', '(S09)', '(S10)', '(S11)', '(S12)', '(S13)', '(S14)', '(S15)', '(S16)', '(S17)', '(S18)', '(S19)', '(S20)', '(S21)', '(S22)', '(S23)', '(S24)', '(S25)', '(S26)', '(S27)', '(S28)', 'Gameplays']


In [3]:
import pandas as pd

In [4]:
import os
print(os.getcwd())


C:\Machine Learning Project Emotion Detection


In [5]:
print(os.listdir())


['.ipynb_checkpoints', 'anaconda_projects', 'Emotion_detection.ipynb', 'gameemo', 'kaggle (1).json', 'S01_all_sessions.csv', 'S01_features.csv', 'S01_filtered.csv', 'S02_all_session.csv', 'S02_features.csv', 'S02_filtered.csv', 'S03_all_session.csv', 'S03_features.csv', 'S03_filtered.csv']


In [6]:
print(os.listdir("gameemo/GAMEEMO/(S01)"))

['Preprocessed EEG Data', 'Raw EEG Data', 'SAM Ratings']


In [7]:
S01_raw_data = "gameemo/GAMEEMO/(S01)/RAW EEG Data"
print(os.listdir(S01_raw_data))

['.csv format', '.mat format']


In [5]:
S01_csv_folder = "gameemo/GAMEEMO/(S01)/RAW EEG Data/.csv format"
print(os.listdir(S01_csv_folder))

['S01G1AllRawChannels.csv', 'S01G2AllRawChannels.csv', 'S01G3AllRawChannels.csv', 'S01G4AllRawChannels.csv']


In [6]:
#(subject1 data)
import pandas as pd
import os
S01_data= pd.DataFrame()
for file in os.listdir(S01_csv_folder):
    if file.endswith(".csv"):
        df=pd.read_csv(os.path.join(S01_csv_folder,file))
        df['session']= file
        S01_data= pd.concat([S01_data , df], ignore_index=True)
        
print(S01_data.head())
        



       AF3      AF4       F3       F4       F7       F8      FC5     FC6  \
0 -35.1282 -16.1538 -44.8718   1.7949  44.8716  -1.7949  -5.8975 -3.8462   
1 -32.5642 -22.3079 -47.9487 -11.0256  40.2561 -16.7949 -20.8975 -2.8206   
2 -27.1794 -13.8459 -39.4871  -3.0769  47.1794  -1.7949  -5.8975 -1.5383   
3 -32.0515 -14.6157 -42.3079  -4.3589  42.3076   3.8462 -11.0259 -2.3076   
4 -27.1794 -13.8462 -40.5127  -5.1279  47.1797   0.0000 -15.8975  0.5127   

        O1       O2       P7      P8       T7       T8  Unnamed: 14  \
0  27.4360 -10.5127   5.8975  7.9487  12.5640  18.2051          NaN   
1  20.2561   2.8206  20.2561  5.8972  19.2307   4.3591          NaN   
2  31.2820  -4.1028  10.2566  9.7434  17.9485  11.2820          NaN   
3  22.3076 -11.0259   3.3330  2.3076   7.4355  10.5127          NaN   
4  19.4873  -2.0513  10.7690  0.0000  11.7949  11.2822          NaN   

                   session  
0  S01G1AllRawChannels.csv  
1  S01G1AllRawChannels.csv  
2  S01G1AllRawChannels.csv  


In [7]:
# Save CSV file
S01_data.to_csv("S01_all_sessions.csv", index=False)
print("saved")

saved


In [8]:
# Remove unwanted columns
S01_data = S01_data.loc[:, ~S01_data.columns.str.contains('^Unnamed')]

#after removing
print(S01_data.columns)


Index(['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7',
       'P8', 'T7', 'T8', 'session'],
      dtype='object')


In [9]:
S01_data= S01_data.dropna()

#print remaining rows
print(S01_data.shape)

(153008, 15)


In [10]:
import numpy as np
from scipy.signal import butter,lfilter, welch
import matplotlib.pyplot as plt

def bandpass_filter(data, lowcut , highcut, fs, order=4):
    nyq=0.5*fs
    low= lowcut/nyq
    high= highcut/nyq
    b,a= butter(order,[low,high], btype='band')
    y=lfilter(b,a, data)
    return y

fs= 128
lowcut= 1
highcut= 50
order=4

#applying filter to all channels in subject

filtered_data_S01= S01_data.copy()

for col in filtered_data_S01.columns:
     if np.issubdtype(filtered_data_S01[col].dtype, np.number):
         filtered_data_S01[col] = bandpass_filter(filtered_data_S01[col], lowcut, highcut,fs ,order)

print("filtered data of S01:")
print(filtered_data_S01.head())
    


filtered data of S01:
         AF3        AF4         F3        F4         F7         F8        FC5  \
0 -13.067811  -6.009269 -16.692464  0.667709  16.692389  -0.667709  -2.193890   
1 -33.829437 -18.284522 -45.575792 -2.991995  42.713995  -7.357326 -11.419640   
2 -25.298595 -16.668050 -38.016483 -8.212938  36.122685 -10.797379 -14.282442   
3 -12.584025  -5.542055 -18.636965 -2.562866  24.474500   3.274956  -2.866427   
4 -18.941539  -6.532882 -24.434351 -0.359501  28.363224   7.974845  -5.620841   

        FC6         O1        O2         P7        P8         T7         T8  \
0 -1.430800  10.206286 -3.910761   2.193890  2.956944   4.673851   6.772360   
1 -3.426905  24.495641 -5.449436  11.181037  7.107482  14.920661  12.875573   
2 -1.774708  20.298515  1.696551  15.507543  6.151679  16.797076   4.330111   
3 -0.146907  15.758325 -3.575570   2.789584  3.436284   7.021334   4.281261   
4 -0.480643  12.750508 -8.994319  -1.444839 -1.080098   1.402993   9.517226   

                

In [12]:
filtered_data_S01.to_csv("S01_filtered.csv", index=False)
print("Filtered data saved as 'S01_filtered.csv'")

Filtered data saved as 'S01_filtered.csv'


In [13]:
S01_filtered = pd.read_csv("S01_filtered.csv") #roading the filtered data of S01

bands={
    'alpha': (8,13),
    'beta' : (13,30),
    'gamma':(30,45)
}

fs=128
features= {}

for col in S01_filtered.columns:
    if np.issubdtype(S01_filtered[col].dtype, np.number):
        f, psd =welch(S01_filtered[col], fs,nperseg=256)

        for band, (low, high) in bands.items():
             freq_range =np.logical_and(f>= low,f<=high)
             power =np.trapezoid(psd[freq_range], f[freq_range])
             features[f'{col}_{band}'] = power

features_df =pd.DataFrame([features])
print("Extracted features Alpha, Beta,Gamma:")
print(features_df.head())

features_df.to_csv("S01_features.csv", index= False)
print("Features saved as 'S01_features.csv'")

Extracted features Alpha, Beta,Gamma:
   AF3_alpha   AF3_beta  AF3_gamma  AF4_alpha   AF4_beta  AF4_gamma  F3_alpha  \
0   4.944704  26.733301  14.389556   4.888976  23.250285  12.363247  5.439044   

     F3_beta   F3_gamma  F4_alpha  ...  P7_gamma  P8_alpha    P8_beta  \
0  29.261755  11.834752  7.049364  ...  7.020157  8.555283  58.236883   

    P8_gamma  T7_alpha    T7_beta   T7_gamma  T8_alpha    T8_beta  T8_gamma  
0  11.899902  7.052466  41.031641  15.221367  6.477936  33.006479   15.2658  

[1 rows x 42 columns]
Features saved as 'S01_features.csv'


In [14]:
#subject 2 
print(os.listdir("gameemo\GAMEEMO\(S02)"))
S02_raw_data = "gameemo/GAMEEMO/(S02)/Raw EEG Data"
print(os.listdir(S02_raw_data))


['Preprocessed EEG Data', 'Raw EEG Data', 'SAM Ratings']
['.csv format', '.mat format']


In [15]:
#subject 2 raw data extract in csv

S02_csv_folder= "gameemo/GAMEEMO/(S02)/Raw EEG data/.csv format"
print(os.listdir(S02_csv_folder))

S02_data= pd.DataFrame()
for file in os.listdir(S02_csv_folder):
    if file.endswith(".csv"):
        df= pd.read_csv(os.path.join(S02_csv_folder, file))
        df['session'] =file
        S02_data= pd.concat([S02_data, df], ignore_index =True)

print(S02_data.head())

['S02G1AllRawChannels.csv', 'S02G2AllRawChannels.csv', 'S02G3AllRawChannels.csv', 'S02G4AllRawChannels.csv']
       AF3      AF4       F3      F4      F7       F8      FC5      FC6  \
0  -4.1025  -9.7437  5.12790   0.000  0.5127  -6.6670 -15.3848 -4.10250   
1   4.1025 -24.1025 -9.87210 -15.000 -3.5898 -21.6670 -30.2563  7.17970   
2 -10.7693 -15.8977  2.56420   0.000  1.0256  -8.2053 -15.2563 -6.66670   
3   4.2307 -14.6155  0.25659  -7.949  3.8459 -23.2053 -15.1282  0.76929   
4   1.0256 -23.5896 -4.10230 -22.949  1.5383 -35.3845 -13.3333  5.12820   

         O1       O2       P7      P8       T7      T8  Unnamed: 14  \
0  14.87160 -0.51318   6.1538  0.0000  2.05130  0.5127          NaN   
1  -0.12842  4.10250   3.5898  7.1797  4.10250  6.6670          NaN   
2  14.87160 -3.07690  12.3074 -1.0256  7.69210  3.0769          NaN   
3  -0.12842 -0.76929   2.8206  4.3591 -0.25659  4.8718          NaN   
4  -7.17940 -1.02560   7.1794  8.2053  3.59010  5.1282          NaN   

             

In [16]:
S02_data.to_csv("S02_all_session.csv", index =False)
print("saved ")

#removing unwanted columns
S02_data =S02_data.loc[: ,~S02_data.columns.str.contains('^Unnamed')]

print(S02_data.columns)

saved 
Index(['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7',
       'P8', 'T7', 'T8', 'session'],
      dtype='object')


In [17]:
#removing missing value rows

S02_data = S02_data.dropna()

print(S02_data.shape)

(153008, 15)


In [18]:
import numpy as np
from scipy.signal import butter,lfilter, welch
import matplotlib.pyplot as plt

def bandpass_filter(data, lowcut , highcut, fs, order=4):
    nyq=0.5*fs
    low= lowcut/nyq
    high= highcut/nyq
    b,a= butter(order,[low,high], btype='band')
    y=lfilter(b,a, data)
    return y

fs= 128
lowcut= 1
highcut= 50
order=4

#applying filter to all channels in subject

filtered_data_S02= S02_data.copy()

for col in filtered_data_S02.columns:
     if np.issubdtype(filtered_data_S02[col].dtype, np.number):
         filtered_data_S02[col] = bandpass_filter(filtered_data_S02[col], lowcut, highcut,fs ,order)

print("filtered data of S02:")
print(filtered_data_S02.head())

filtered data of S02:
        AF3        AF4        F3        F4        F7         F8        FC5  \
0 -1.526144  -3.624690  1.907596  0.000000  0.190726  -2.480147  -5.723198   
1 -1.009927 -14.989543 -0.502506 -5.580052 -1.018479 -12.181584 -20.765976   
2 -0.892908 -19.442632 -5.870328 -9.272654 -1.909744 -15.508367 -22.214442   
3 -4.310798  -8.667107  1.382277 -0.846498  2.401084  -8.462357  -5.739254   
4  2.746808  -8.629091  2.039400 -8.515211  4.038537 -19.899087  -3.767147   

        FC6        O1        O2        P7        P8        T7        T8  \
0 -1.526144  5.532286 -0.190905  2.289235  0.000000  0.763091  0.190726   
1  0.134802  9.145507  1.208908  5.139555  2.670873  2.794210  2.797086   
2  2.535527  3.360409  1.463660  5.931663  4.056798  5.108933  5.193864   
3 -3.495296  4.270043 -2.596621  6.127403 -0.022613  3.407404  2.607617   
4  0.543415 -3.294611 -1.826050  2.124542  3.528904 -1.047636  2.344531   

                   session  
0  S02G1AllRawChannels.csv  


In [19]:
filtered_data_S02.to_csv("S02_filtered.csv", index=False)
print("Filtered data saved as 'S02_filtered.csv'")

Filtered data saved as 'S02_filtered.csv'


In [20]:
S02_filtered = pd.read_csv("S02_filtered.csv") #roading the filtered data of S01

bands={
    'alpha': (8,13),
    'beta' : (13,30),
    'gamma':(30,45)
}

fs=128
features= {}

for col in S02_filtered.columns:
    if np.issubdtype(S02_filtered[col].dtype, np.number):
        f, psd =welch(S02_filtered[col], fs,nperseg=256)

        for band, (low, high) in bands.items():
             freq_range =np.logical_and(f>= low,f<=high)
             power =np.trapezoid(psd[freq_range], f[freq_range])
             features[f'{col}_{band}'] = power

features_df =pd.DataFrame([features])
print("Extracted features Alpha, Beta,Gamma:")
print(features_df.head())

features_df.to_csv("S02_features.csv", index= False)
print("Features saved as 'S02_features.csv'")

Extracted features Alpha, Beta,Gamma:
   AF3_alpha    AF3_beta  AF3_gamma  AF4_alpha   AF4_beta  AF4_gamma  \
0  13.901486  105.820331   5.612854  14.505568  83.082507  14.848187   

   F3_alpha    F3_beta   F3_gamma  F4_alpha  ...  P7_gamma   P8_alpha  \
0  14.38749  77.214804  16.753008  19.54383  ...  6.313043  14.362387   

      P8_beta  P8_gamma   T7_alpha    T7_beta  T7_gamma   T8_alpha  \
0  109.417144   5.29953  12.352115  85.005658  11.62595  13.422989   

      T8_beta  T8_gamma  
0  109.973121  5.526934  

[1 rows x 42 columns]
Features saved as 'S02_features.csv'


In [21]:
#subject 3
print(os.listdir("gameemo\GAMEEMO\(S03)"))
S03_raw_data = "gameemo/GAMEEMO/(S03)/Raw EEG Data"
print(os.listdir(S03_raw_data))

['Preprocessed EEG Data', 'Raw EEG Data', 'SAM Ratings']
['.csv format', '.mat format']


In [22]:
#subject 3 raw data extract in csv

S03_csv_folder= "gameemo/GAMEEMO/(S03)/Raw EEG data/.csv format"
print(os.listdir(S03_csv_folder))

S03_data= pd.DataFrame()
for file in os.listdir(S03_csv_folder):
    if file.endswith(".csv"):
        df= pd.read_csv(os.path.join(S03_csv_folder, file))
        df['session'] =file
        S03_data= pd.concat([S03_data, df], ignore_index =True)

print(S03_data.head())

['S03G1AllRawChannels.csv', 'S03G2AllRawChannels.csv', 'S03G3AllRawChannels.csv', 'S03G4AllRawChannels.csv']
       AF3      AF4       F3       F4        F7        F8      FC5      FC6  \
0   2.0513  -2.0513  48.7183 -20.5127  109.7437 -257.9485  -90.769  13.3335   
1  17.0513 -17.0513  44.8717 -35.5127   94.7437 -250.5128 -105.769  28.3335   
2  32.0513 -32.0513  46.6667 -46.6667   86.1536 -235.5128 -120.769  43.3335   
3  47.0513 -43.0769  43.0769 -60.0000  101.1536 -220.5128 -135.769  58.3335   
4  62.0513 -58.0769  58.0769 -63.8462  100.7690 -235.5128 -141.282  73.3335   

         O1       O2        P7        P8        T7       T8  Unnamed: 14  \
0 -128.2051  20.5132  -56.4102 -156.4102   82.5645  76.9233          NaN   
1 -143.2051  35.5132  -71.4102 -171.4102   70.5128  62.3077          NaN   
2 -158.2051  50.5132  -86.4102 -186.4102   85.5128  74.8718          NaN   
3 -173.2051  65.5132 -101.4102 -171.4102  100.5128  89.8718          NaN   
4 -188.2051  80.5132  -96.1538 -186.

In [23]:
S03_data.to_csv("S03_all_session.csv", index =False)
print("saved ")

#removing unwanted columns
S03_data =S03_data.loc[: ,~S03_data.columns.str.contains('^Unnamed')]

print(S03_data.columns)

saved 
Index(['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7',
       'P8', 'T7', 'T8', 'session'],
      dtype='object')


In [24]:
#removing missing value rows

S03_data = S03_data.dropna()

print(S03_data.shape)

(153008, 15)


In [25]:
import numpy as np
from scipy.signal import butter,lfilter, welch
import matplotlib.pyplot as plt

def bandpass_filter(data, lowcut , highcut, fs, order=4):
    nyq=0.5*fs
    low= lowcut/nyq
    high= highcut/nyq
    b,a= butter(order,[low,high], btype='band')
    y=lfilter(b,a, data)
    return y

fs= 128
lowcut= 1
highcut= 50
order=4

#applying filter to all channels in subject

filtered_data_S03= S03_data.copy()

for col in filtered_data_S03.columns:
     if np.issubdtype(filtered_data_S03[col].dtype, np.number):
         filtered_data_S03[col] = bandpass_filter(filtered_data_S03[col], lowcut, highcut,fs ,order)

print("filtered data of S03:")
print(filtered_data_S03.head())

filtered data of S03:
         AF3        AF4         F3         F4          F7          F8  \
0   0.763091  -0.763091  18.123375  -7.630795   40.825034  -95.957729   
1   7.611209  -7.611209  46.808955 -25.891324  103.086004 -252.649431   
2  22.175288 -22.175288  38.243971 -36.427089   75.176446 -206.178461   
3  34.242451 -32.763961  22.528545 -39.421960   41.445439 -107.493448   
4  42.256512 -38.321139  31.835432 -44.666844   67.831173 -134.475796   

         FC5        FC6          O1         O2         P7          P8  \
0 -33.766380   4.960108  -47.692738   7.630981 -20.984788  -58.185132   
1 -95.457732  18.782621 -132.526223  25.891819 -61.436323 -160.454358   
2 -97.538892  31.759252 -129.339980  37.858239 -68.351898 -153.299551   
3 -80.413295  40.113982  -99.895949  43.850472 -62.532142 -103.414465   
4 -93.880955  49.270382 -120.683265  53.733827 -68.514669 -108.512280   

          T7         T8                  session  
0  30.714278  28.615732  S03G1AllRawChannels.csv 

In [26]:
filtered_data_S03.to_csv("S03_filtered.csv", index=False)
print("Filtered data saved as 'S03_filtered.csv'")

Filtered data saved as 'S03_filtered.csv'


In [27]:
S03_filtered = pd.read_csv("S03_filtered.csv") #roading the filtered data of S03

bands={
    'alpha': (8,13),
    'beta' : (13,30),
    'gamma':(30,45)
}

fs=128
features= {}

for col in S03_filtered.columns:
    if np.issubdtype(S03_filtered[col].dtype, np.number):
        f, psd =welch(S03_filtered[col], fs,nperseg=256)

        for band, (low, high) in bands.items():
             freq_range =np.logical_and(f>= low,f<=high)
             power =np.trapezoid(psd[freq_range], f[freq_range])
             features[f'{col}_{band}'] = power

features_df =pd.DataFrame([features])
print("Extracted features Alpha, Beta,Gamma:")
print(features_df.head())

features_df.to_csv("S03_features.csv", index= False)
print("Features saved as 'S03_features.csv'")

Extracted features Alpha, Beta,Gamma:
   AF3_alpha   AF3_beta  AF3_gamma  AF4_alpha   AF4_beta  AF4_gamma  F3_alpha  \
0  16.471641  60.110814   6.656928   9.918349  78.088374   9.119685  9.496943   

   F3_beta  F3_gamma  F4_alpha  ...   P7_gamma  P8_alpha     P8_beta  \
0  79.2996  8.770764  17.13726  ...  11.284742   19.1924  101.419576   

    P8_gamma   T7_alpha    T7_beta  T7_gamma   T8_alpha    T8_beta  T8_gamma  
0  12.311619  14.581443  77.020808   9.93544  11.626656  66.685717  10.82213  

[1 rows x 42 columns]
Features saved as 'S03_features.csv'


In [28]:
# subject 4

print(os.listdir("gameemo\GAMEEMO\(S04)"))
S04_raw_data = "gameemo/GAMEEMO/(S04)/Raw EEG Data"
print(os.listdir(S04_raw_data))

['Preprocessed EEG Data', 'Raw EEG Data', 'SAM Ratings']
['.csv format', '.mat format']


In [29]:
#subject 4 raw data extract in csv

S04_csv_folder= "gameemo/GAMEEMO/(S04)/Raw EEG data/.csv format"
print(os.listdir(S04_csv_folder))

S04_data= pd.DataFrame()
for file in os.listdir(S04_csv_folder):
    if file.endswith(".csv"):
        df= pd.read_csv(os.path.join(S04_csv_folder, file))
        df['session'] =file
        S04_data= pd.concat([S04_data, df], ignore_index =True)

print(S04_data.head())

['S04G1AllRawChannels.csv', 'S04G2AllRawChannels.csv', 'S04G3AllRawChannels.csv', 'S04G4AllRawChannels.csv']
       AF3      AF4       F3       F4       F7        F8       FC5     FC6  \
0   9.2310  4.61520 -38.9746 -20.5127  37.9487 -103.0769  24.61520 -4.6152   
1  12.3074  2.56420 -32.3079 -24.6155  22.9487 -118.0769   9.61520 -2.5642   
2   6.4102  0.25635 -42.8203 -30.0000   7.9487 -118.7178  -0.25635 -5.3843   
3  17.9487  7.17970 -31.7949 -15.8975  12.8208 -117.4358  -3.58980  3.5898   
4  11.2820  1.02560 -28.7180 -18.9744  10.7693 -118.4617  -1.02560  1.5383   

        O1       O2       P7       P8       T7       T8  Unnamed: 14  \
0 -10.2563 -25.6411  29.7437  34.3589   6.1538 -25.6411          NaN   
1  -8.7180 -21.5383  30.2561  31.2820   8.2048 -25.6409          NaN   
2   3.8462 -21.2817  39.2310  43.8462   1.7949 -28.4614          NaN   
3  -5.6411 -16.4102  34.8721  44.1025  14.3589 -25.1279          NaN   
4  -7.1794 -15.8977  35.3845  45.6409  14.8718 -25.6414       

In [30]:
S04_data.to_csv("S04_all_session.csv", index =False)
print("saved ")

#removing unwanted columns
S04_data =S04_data.loc[: ,~S04_data.columns.str.contains('^Unnamed')]

print(S04_data.columns)

saved 
Index(['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7',
       'P8', 'T7', 'T8', 'session'],
      dtype='object')


In [31]:
#removing missing value rows

S04_data = S04_data.dropna()

print(S04_data.shape)

(153008, 15)


In [33]:
import numpy as np
from scipy.signal import butter,lfilter, welch
import matplotlib.pyplot as plt

def bandpass_filter(data, lowcut , highcut, fs, order=4):
    nyq=0.5*fs
    low= lowcut/nyq
    high= highcut/nyq
    b,a= butter(order,[low,high], btype='band')
    y=lfilter(b,a, data)
    return y

fs= 128
lowcut= 1
highcut= 50
order=4

#applying filter to all channels in subject

filtered_data_S04= S04_data.copy()

for col in filtered_data_S04.columns:
     if np.issubdtype(filtered_data_S04[col].dtype, np.number):
         filtered_data_S04[col] = bandpass_filter(filtered_data_S04[col], lowcut, highcut,fs ,order)

print("filtered data of S04:")
print(filtered_data_S04.head())

filtered data of S04:
         AF3       AF4         F3         F4         F7          F8  \
0   3.433964  1.716870 -14.498685  -7.630795  14.117047  -38.344961   
1  10.284786  3.806901 -36.111848 -21.837528  31.996006 -107.644773   
2   8.693930  1.031116 -30.417422 -23.490622  11.803773 -102.652530   
3   5.870436  0.949908 -20.927807 -14.245987  -6.032991  -66.543566   
4  11.275349  4.407322 -17.628335  -6.642049   7.104366  -71.543426   

         FC5       FC6        O1         O2         P7         P8         T7  \
0   9.156939 -1.716870 -3.815379  -9.538577  11.064759  12.781629   2.289235   
1  18.793436 -3.806901 -9.583334 -23.863058  29.642242  32.876890   6.856351   
2   2.385079 -2.938731 -2.515366 -17.623545  29.112637  30.814319   4.873864   
3 -10.946470 -0.113597  4.880638  -7.792723  23.179859  27.803619   3.271757   
4  -3.236168  3.925436 -4.868922  -8.572961  22.256086  31.257866  12.079425   

          T8                  session  
0  -9.538577  S04G1AllRawChann

In [34]:
filtered_data_S04.to_csv("S04_filtered.csv", index=False)
print("Filtered data saved as 'S04_filtered.csv'")

Filtered data saved as 'S04_filtered.csv'


In [35]:
S04_filtered = pd.read_csv("S04_filtered.csv") #roading the filtered data of S01

bands={
    'alpha': (8,13),
    'beta' : (13,30),
    'gamma':(30,45)
}

fs=128
features= {}

for col in S04_filtered.columns:
    if np.issubdtype(S04_filtered[col].dtype, np.number):
        f, psd =welch(S04_filtered[col], fs,nperseg=256)

        for band, (low, high) in bands.items():
             freq_range =np.logical_and(f>= low,f<=high)
             power =np.trapezoid(psd[freq_range], f[freq_range])
             features[f'{col}_{band}'] = power

features_df =pd.DataFrame([features])
print("Extracted features Alpha, Beta,Gamma:")
print(features_df.head())

features_df.to_csv("S04_features.csv", index= False)
print("Features saved as 'S04_features.csv'")

Extracted features Alpha, Beta,Gamma:
   AF3_alpha  AF3_beta  AF3_gamma  AF4_alpha  AF4_beta  AF4_gamma  F3_alpha  \
0   4.009365  26.11234   2.946087   4.297493  62.41736   4.686405  3.910053   

     F3_beta  F3_gamma  F4_alpha  ...  P7_gamma   P8_alpha     P8_beta  \
0  45.748699  4.001677  3.871675  ...  8.400027  19.355581  100.192985   

    P8_gamma  T7_alpha    T7_beta  T7_gamma  T8_alpha    T8_beta  T8_gamma  
0  10.010727  3.816513  47.510963  4.312599  4.748489  67.314675  5.447689  

[1 rows x 42 columns]
Features saved as 'S04_features.csv'
